## **Load Llama3 quantised model with mlx** ##

In [1]:
from mlx_lm import load, generate
      
model, tokenizer = load("mlx-community/Meta-Llama-3-8B-Instruct-4bit")

/Users/user/anaconda3/envs/splore/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 53092.46it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## **Load Data** ##

In [99]:
import pandas as pd
import json, os

df = pd.read_csv('../../data/data.csv')

df = df.dropna()
print(df.isnull().sum())

df.head(3)

conversation_id    0
message            0
sentiment          0
dtype: int64


,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpful...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper


### **Set Prompt** ###

In [88]:
message = df.message[0]
ground_truth = df.sentiment[0]
y_labels = df.sentiment.unique().tolist()


output_format = '{"sentiment": "<label>"}'
prompt = f"""
            <|begin_of_text|><|start_header_id|>user<|end_header_id|>
            INSTRUCTIONS:
                Given the message: '{message}', map the sentiment to one of the following: {y_labels} based on the context semantic meaning.
                Only output the sentiment label in json format.
                {output_format}
            <|eot_id|><|start_header_id|>assistant<|end_header_id|>
        """

print(message)
print(ground_truth)
print(prompt)
response = generate(model, tokenizer, prompt, max_tokens=2048, temp=0.0, verbose=False)

print(response)

Are you a fan of Google or Microsoft?
Curious to dive deeper

            <|begin_of_text|><|start_header_id|>user<|end_header_id|>
            INSTRUCTIONS:
                Given the message: 'Are you a fan of Google or Microsoft?', map the sentiment to one of the following: ['Curious to dive deeper', 'Happy', 'Neutral', 'Surprised', 'Disgusted', 'Sad', 'Fearful', 'Angry'] based on the context semantic meaning.
                Only output the sentiment label in json format.
                {"sentiment": "<label>"}
            <|eot_id|><|start_header_id|>assistant<|end_header_id|>
        
 {"sentiment": "Curious to dive deeper"}


In [89]:
message = df.message[1]
ground_truth = df.sentiment[1]

y_labels = df.sentiment.unique().tolist()

def set_prompt(message, y_labels) -> str:
    output_format = '{"sentiment": "<label>"}'
    prompt = f"""
                <|begin_of_text|><|start_header_id|>user<|end_header_id|>
                INSTRUCTIONS:
                    Given the message: '{message}', map the sentiment to one of the following: {y_labels} based on the context semantic meaning.
                    Your response needs to be one of the element in the list: {y_labels}.
                    Only output the sentiment label in json format.
                    {output_format}
                <|eot_id|><|start_header_id|>assistant<|end_header_id|>
            """
    return prompt

In [91]:
def get_sentiment(prompt, model, tokenizer):
    response = generate(model, tokenizer, prompt, max_tokens=2048, temp=0.0, verbose=False)
    return response

In [90]:
prompt = set_prompt(message, y_labels)
response = get_sentiment(prompt, model, tokenizer)

# response = json.loads(response)

print(f"Message: {message}")
print(f"Ground Truth: {ground_truth}")
print(f"Predicted Sentiment: {response}")

Message: Both are excellent technology they are helpful in many ways. For the security purpose both are super.
Ground Truth: Curious to dive deeper
Predicted Sentiment:  {"sentiment": "Happy"}


In [111]:
import json
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import os
from json import JSONDecodeError

def set_prompt(message, y_labels) -> str:
    output_format = '{"sentiment": "<label>"}'
    prompt = f"""
                <|begin_of_text|><|start_header_id|>user<|end_header_id|>
                INSTRUCTIONS:
                    Given the message: '{message}', map the sentiment to one of the following: {y_labels} based on the context semantic meaning.
                    Your response needs to be one of the element in the list: {y_labels}.
                    Only output the sentiment label in json format.
                    {output_format}
                <|eot_id|><|start_header_id|>assistant<|end_header_id|>
            """
    return prompt

def get_sentiment(prompt, model, tokenizer):
    return generate(model, tokenizer, prompt, max_tokens=2048, temp=0.0, verbose=False)


for index, row in df.iterrows():
    # if index == 10:
    #     break
    message = row['message']
    ground_truth = row['sentiment']
    prompt = set_prompt(message, y_labels)
    response = get_sentiment(prompt, model, tokenizer)
    try:
        response = json.loads(response)
        response = response['sentiment']
    except JSONDecodeError:
        response = response
    
    print(f"{index} | {response} | {ground_truth}")



0 | Curious to dive deeper | Curious to dive deeper
1 | Happy | Curious to dive deeper
2 | Neutral | Curious to dive deeper
3 | Neutral | Curious to dive deeper
4 | Neutral | Curious to dive deeper
5 | Happy | Curious to dive deeper
6 | Surprised | Curious to dive deeper
7 | Curious to dive deeper | Curious to dive deeper
8 | Happy | Curious to dive deeper
9 | Curious to dive deeper | Curious to dive deeper
10 | Curious to dive deeper | Curious to dive deeper
11 | Curious to dive deeper | Curious to dive deeper
12 | Curious to dive deeper | Curious to dive deeper
13 | Curious to dive deeper | Happy
14 | Surprised | Curious to dive deeper
15 | Neutral | Curious to dive deeper
16 | Neutral | Curious to dive deeper
17 | Happy | Curious to dive deeper
18 | Neutral | Curious to dive deeper
19 | Curious to dive deeper | Curious to dive deeper
20 | Happy | Curious to dive deeper
21 | Curious to dive deeper | Curious to dive deeper
22 | Curious to dive deeper | Curious to dive deeper
23 | Surp

KeyboardInterrupt: 

In [129]:
import json
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from json import JSONDecodeError

def set_prompt(message, y_labels) -> str:
    output_format = '{"sentiment": "<label>"}'
    prompt = f"""
                <|begin_of_text|><|start_header_id|>user<|end_header_id|>
                INSTRUCTIONS:
                    Given the message: '{message}', map the sentiment to one of the following: {y_labels} based on the context semantic meaning.
                    Your response needs to be one of the element in the list: {y_labels}.
                    Only output the sentiment label in json format.
                    {output_format}
                <|eot_id|><|start_header_id|>assistant<|end_header_id|>
            """
    return prompt

def get_sentiment(prompt, model, tokenizer):
    return generate(model, tokenizer, prompt, max_tokens=2048, temp=0.0, verbose=False)

def evaluate_and_save_batches(df, y_labels, model, tokenizer, batch_size=1000, output_file="predictions.csv"):
    num_batches = len(df) // batch_size + (1 if len(df) % batch_size != 0 else 0)
    
    for i in range(num_batches):
        batch = df.iloc[i * batch_size : (i + 1) * batch_size]
        ground_truths = batch['sentiment'].tolist()
        indices = batch.index.tolist()
        messages = batch['message'].tolist()
        predictions = []

        for idx, message in enumerate(batch['message']):
            prompt = set_prompt(message, y_labels)
            response = get_sentiment(prompt, model, tokenizer)
            
            try:
                response = json.loads(response)
                response = response['sentiment']
            except JSONDecodeError:
                response = response 
            
            predictions.append(response)
            print(f"{idx} | {response} | {ground_truth}")

        batch_df = pd.DataFrame({'index': indices, 'message':messages, 'y_true': ground_truths, 'y_pred': predictions})
        
        if i == 0:
            batch_df.to_csv(output_file, index=False, mode='w')
        else:
            batch_df.to_csv(output_file, index=False, mode='a', header=False)
        
        print(f"Processed and saved batch {i + 1}/{num_batches}")

def load_and_evaluate(output_file="predictions.csv"):
    combined_df = pd.read_csv(output_file)
    y_true = combined_df['y_true']
    y_pred = combined_df['y_pred']
    y_labels = y_true.unique().tolist()

    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=y_labels)
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=y_labels, yticklabels=y_labels)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

    print("Classification Report:\n")
    print(classification_report(y_true, y_pred, target_names=y_labels))


In [130]:

evaluate_and_save_batches(df[:50], y_labels, model, tokenizer, batch_size=10, output_file="predictions.csv")


0 | Curious to dive deeper | Curious to dive deeper
1 | Happy | Curious to dive deeper
2 | Neutral | Curious to dive deeper
3 | Neutral | Curious to dive deeper
4 | Neutral | Curious to dive deeper
5 | Happy | Curious to dive deeper
6 | Surprised | Curious to dive deeper
7 | Curious to dive deeper | Curious to dive deeper
8 | Happy | Curious to dive deeper
9 | Curious to dive deeper | Curious to dive deeper
Processed and saved batch 1/5
0 | Curious to dive deeper | Curious to dive deeper
1 | Curious to dive deeper | Curious to dive deeper
2 | Curious to dive deeper | Curious to dive deeper
3 | Curious to dive deeper | Curious to dive deeper
4 | Surprised | Curious to dive deeper
5 | Neutral | Curious to dive deeper
6 | Neutral | Curious to dive deeper
7 | Happy | Curious to dive deeper
8 | Neutral | Curious to dive deeper
9 | Curious to dive deeper | Curious to dive deeper
Processed and saved batch 2/5
0 | Happy | Curious to dive deeper
1 | Curious to dive deeper | Curious to dive deep